In [1]:
import getpass
import os
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [2]:
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
llm = ChatGroq(temperature=0, model="llama3-70b-8192")

In [4]:
from operator import itemgetter
from typing import Dict, List

from langchain_core.messages import AIMessage
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.tools import tool

In [18]:
@tool
def count_emails(last_n_days: int) -> int:
    """Multiplica dos números enteros."""
    return last_n_days * 2


@tool
def send_email(message: str, recipient: str) -> str:
    """Email para enviar."""
    return f"Successfully sent email to {recipient}."

In [19]:
tools = [count_emails, send_email]

In [20]:
llm_with_tools = llm.bind_tools(tools)

In [21]:
def call_tools(msg: AIMessage) -> List[Dict]:
    """Simple herramienta secuencial de ayuda a la llamada."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])
    return tool_calls

In [22]:
chain = llm_with_tools | call_tools
chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A9230F2E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A923173460>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'count_emails', 'description': 'count_emails(last_n_days: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'last_n_days': {'type': 'integer'}}, 'required': ['last_n_days']}}}, {'type': 'function', 'function': {'name': 'send_email', 'description': 'send_email(message: str, recipient: str) -> str - Email para enviar.', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string'}, 'recipient': {'type': 'string'}}, 'required': ['message', 'recipient']}}}]})
| RunnableLambda(call_tools)

In [23]:
resultado = chain.invoke("¿cuántos correos he recibido en los últimos 5 días?")
resultado

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_p9r9',
  'output': 10}]

# Aprobación humana

In [24]:
import json

In [25]:
def human_approval(msg: AIMessage) -> Runnable:
    tool_strs = "\n\n".join(
        json.dumps(tool_call, indent=2) for tool_call in msg.tool_calls
    )
    input_msg = (
        f"Aprueba las siguientes invocaciones de herramientas\n\n{tool_strs}\n\n"
        "Todo lo que no sea Y/Yes (case-insensitive) se tratará como un no."
    )
    resp = input(input_msg)
    if resp.lower() not in ("yes", "y"):
        raise ValueError(f"Invocaciones de herramientas no aprobadas:\n\n{tool_strs}")
    return msg

In [26]:
chain = llm_with_tools | human_approval | call_tools
chain

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A9230F2E90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A923173460>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'count_emails', 'description': 'count_emails(last_n_days: int) -> int - Multiplica dos números enteros.', 'parameters': {'type': 'object', 'properties': {'last_n_days': {'type': 'integer'}}, 'required': ['last_n_days']}}}, {'type': 'function', 'function': {'name': 'send_email', 'description': 'send_email(message: str, recipient: str) -> str - Email para enviar.', 'parameters': {'type': 'object', 'properties': {'message': {'type': 'string'}, 'recipient': {'type': 'string'}}, 'required': ['message', 'recipient']}}}]})
| RunnableLambda(human_approval)
| RunnableLambda(call_tools)

In [28]:
respuesta = chain.invoke("¿cuántos correos he recibido en los últimos 5 días?")
respuesta

[{'name': 'count_emails',
  'args': {'last_n_days': 5},
  'id': 'call_c73f',
  'output': 10}]

In [29]:
respuesta = chain.invoke("Send sally@gmail.com an email saying 'What's up homie'")
respuesta

[{'name': 'send_email',
  'args': {'message': "What's up homie", 'recipient': 'sally@gmail.com'},
  'id': 'call_c9y8',
  'output': 'Successfully sent email to sally@gmail.com.'}]